# Symmetry Reduction in the Shortest Path Problem: ILP Formulation

This notebook presents an Integer Linear Programming formulation for the shortest path problem and investigates the role of graph symmetries. We demonstrate how automorphisms generate equivalent optimal solutions and how lexicographic symmetry-breaking constraints can eliminate redundant solutions while preserving optimality.

---

## *Mathematical Background*

### **Shortest Path as Integer Linear Program**

Given an undirected graph $G = (V, E)$ with edge weights $w_e > 0$, a source node $s \in V$, and a target node $t \in V$, the shortest path problem seeks a path from $s$ to $t$ that minimizes the total edge weight.<sup>[[1]](#ref1), [[2]](#ref2)</sup>

To formulate this as an Integer Linear Program with flow conservation constraints, we model the problem on a directed graph $D = (V, A)$. We replace each undirected edge $\{u, v\} \in E$ with two directed arcs $(u, v)$ and $(v, u)$ in $A$, both inheriting the weight $w_{\{u,v\}}$.

We introduce binary decision variables $x_{uv} \in \{0, 1\}$ for each arc $(u, v) \in A$, where $x_{uv} = 1$ indicates that the arc from $u$ to $v$ is selected.

The objective function minimizes the total weight:

\begin{equation*}
\min \sum_{(u,v) \in A} w_{uv} \cdot x_{uv}
\end{equation*}

We impose flow conservation constraints to ensure a valid path from $s$ to $t$.<sup>[[1]](#ref1)</sup> Let $\delta^+(v)$ denote the set of arcs leaving node $v$ and $\delta^-(v)$ the set of arcs entering node $v$. The constraints are:

\begin{equation*}
\sum_{(v,u) \in \delta^+(v)} x_{vu} - \sum_{(u,v) \in \delta^-(v)} x_{uv} = b_v, \quad \forall v \in V
\end{equation*}

where the net flow $b_v$ is defined as:

\begin{equation*}
b_v = 
\begin{cases} 
+1 & \text{if } v = s \\[4pt]
-1 & \text{if } v = t \\[4pt]
\phantom{+}0 & \text{otherwise}
\end{cases}
\end{equation*}

This formulation guarantees that exactly one unit of flow travels from $s$ to $t$. Note that since $w_e > 0$, no cycles will be formed in an optimal solution.

---

### **Graph Symmetries**

A fundamental concept in combinatorial optimization is that of structural symmetry.<sup>[[3]](#ref3), [[4]](#ref4)</sup> Many optimization problems exhibit symmetries that create multiple equivalent solutions, inflating the search space without contributing distinct optima. Understanding and exploiting these symmetries can significantly reduce computational effort.

A *graph automorphism* is a permutation of the vertex set that preserves the edge structure.<sup>[[5]](#ref5)</sup> Formally, let $G = (V, E)$ be an undirected graph. A bijection $\pi: V \to V$ is an automorphism of $G$ if and only if:

\begin{equation*}
\{u, v\} \in E \iff \{\pi(u), \pi(v)\} \in E, \quad \forall u, v \in V
\end{equation*}

For weighted graphs $G = (V, E, w)$ where $w: E \to \mathbb{R}^+$ assigns weights to edges, we additionally require that the automorphism preserves edge weights:

\begin{equation*}
w(\{u, v\}) = w(\{\pi(u), \pi(v)\}), \quad \forall \{u, v\} \in E
\end{equation*}

The set of all automorphisms of a graph $G$ forms a group under composition, denoted $\mathrm{Aut}(G)$.<sup>[[5]](#ref5)</sup> The size of this group measures the degree of symmetry.

When symmetries exist, the optimization problem inherits them.<sup>[[3]](#ref3), [[4]](#ref4)</sup> Let $\mathcal{F} \subseteq \{0,1\}^{|A|}$ denote the set of feasible solutions. An automorphism $\pi \in \mathrm{Aut}(G)$ induces a permutation on $\mathcal{F}$ such that if $\mathbf{x}$ is a feasible solution, then $\pi(\mathbf{x})$ is also feasible and has the same objective value.

The set of all solutions equivalent to a given solution $\mathbf{x}$ under the automorphism group is called its *orbit*:

\begin{equation*}
\mathrm{Orb}(\mathbf{x}) = \{\pi(\mathbf{x}) : \pi \in \mathrm{Aut}(G)\}
\end{equation*}

The orbits partition the feasible region $\mathcal{F}$ into disjoint equivalence classes. From an optimization perspective, all solutions in the same orbit are interchangeable. Therefore, the goal is to search over the quotient space $\mathcal{F} / \mathrm{Aut}(G)$, finding exactly one representative optimal solution.

---

### **Symmetry Breaking**

To prevent redundant exploration of symmetric solutions, we introduce *symmetry-breaking constraints* that eliminate all but one representative from each equivalence class.<sup>[[3]](#ref3), [[4]](#ref4)</sup>

The most common technique is *lexicographic symmetry breaking*.<sup>[[6]](#ref6)</sup> To break symmetry, we impose an ordering on symmetric elements. Consider two symmetric nodes $a$ and $b$ adjacent to the source $s$, with $a < b$ (based on index). Since flow constraints imply that $\sum_{v} x_{sv} = 1$, we can only select at most one outgoing edge from the source. We add the constraint:

\begin{equation*}
x_{sa} \geq x_{sb}
\end{equation*}

This enforces a lexicographic preference: if the solver chooses to route flow through $b$ (i.e., $x_{sb} = 1$), it requires $x_{sa} \geq 1$. However, since we can pick only one edge, this configuration is infeasible unless $a$ and $b$ are the same node (or both are $0$). Effectively, this constraint forbids the selection of $b$ if the equivalent option $a$ is available but not chosen, forcing the solver to choose the canonical representative $a$.

More generally, for a set of symmetric nodes $\{v_1, v_2, \ldots, v_k\}$ adjacent to the source with $v_1 < v_2 < \cdots < v_k$, we impose the chain of inequalities:

\begin{equation*}
x_{s,v_1} \geq x_{s,v_2} \geq \cdots \geq x_{s,v_k}
\end{equation*}

A valid symmetry-breaking scheme must satisfy two properties:<sup>[[4]](#ref4)</sup>
- *Completeness*: at least one solution from each equivalence class remains feasible.
- *Soundness*: the optimal value is preserved.

Lexicographic constraints satisfy both properties because they retain the lexicographically smallest representative of each orbit.

---

### **References**

<a id="ref1"></a>
<sup>[[1]](#ref1)</sup> Ahuja, R. K., Magnanti, T. L., & Orlin, J. B. (1993). *Network Flows: Theory, Algorithms, and Applications*. Prentice Hall.

<a id="ref2"></a>
<sup>[[2]](#ref2)</sup> Schrijver, A. (2003). *Combinatorial Optimization: Polyhedra and Efficiency*. Springer.

<a id="ref3"></a>
<sup>[[3]](#ref3)</sup> Margot, F. (2010). Symmetry in Integer Linear Programming. In *50 Years of Integer Programming 1958–2008* (pp. 647–686). Springer.

<a id="ref4"></a>
<sup>[[4]](#ref4)</sup> Liberti, L. (2012). Symmetry in Mathematical Programming. In *Mixed Integer Nonlinear Programming* (pp. 263–283). Springer.

<a id="ref5"></a>
<sup>[[5]](#ref5)</sup> Godsil, C., & Royle, G. (2001). *Algebraic Graph Theory*. Springer.

<a id="ref6"></a>
<sup>[[6]](#ref6)</sup> Crawford, J. M., Ginsberg, M. L., Luks, E. M., & Roy, A. (1996). Symmetry-breaking predicates for search problems. In *Proceedings of KR'96* (pp. 148–159).

## *Computational Implementation*

In [1]:
using JuMP
using HiGHS
println("Packages loaded | Julia ", VERSION)

Packages loaded | Julia 1.11.5


In [2]:
function build_sp_model(arcs, weights, N, source, target;
                        symmetry_breaking=false, symmetric_nodes=nothing, fixed_obj=nothing)
    """
    Builds an integer linear programming model for the shortest path problem.
    
    Args:
        arcs: List of arcs (tuples (u,v)) of the graph
        weights: Dictionary with the weight of each arc
        N: Total number of nodes in the graph
        source: Source node
        target: Target/destination node
        symmetry_breaking: (optional) Enables symmetry breaking constraints
        symmetric_nodes: (optional) Symmetric nodes for symmetry breaking
        fixed_obj: (optional) Fixes the objective value (for enumerating optimal solutions)
    
    Returns: (model, x) - The JuMP model and decision variables
    """
    
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    
    @variable(model, x[(u,v) in arcs], Bin)
    @objective(model, Min, sum(weights[(u,v)] * x[(u,v)] for (u,v) in arcs))
    
    for v in 1:N
        flow_in = sum(x[(a,b)] for (a,b) in arcs if b == v; init=0)
        flow_out = sum(x[(a,b)] for (a,b) in arcs if a == v; init=0)
        rhs = (v == source) ? 1 : (v == target) ? -1 : 0
        @constraint(model, flow_out - flow_in == rhs)
    end
    
    if fixed_obj !== nothing
        @constraint(model, sum(weights[(u,v)] * x[(u,v)] for (u,v) in arcs) == fixed_obj)
    end
    
    if symmetry_breaking && symmetric_nodes !== nothing
        sorted_nodes = sort(symmetric_nodes)
        for i in 1:(length(sorted_nodes) - 1)
            a, b = sorted_nodes[i], sorted_nodes[i+1]
            @constraint(model, x[(source, a)] >= x[(source, b)])
        end
    end
    
    return model, x
end

function extract_path(x, arcs, source, target)
    """
    Extracts the solution path from the binary decision variables.
    
    Args:
        x: Model decision variables (selected arcs)
        arcs: List of graph arcs
        source: Source node
        target: Target node
    
    Returns: Vector with the sequence of nodes forming the path
    """
    
    selected = [(u,v) for (u,v) in arcs if value(x[(u,v)]) > 0.5]
    path, current, visited = [source], source, Set([source])
    
    while current != target
        found = false
        for (u, v) in selected
            if u == current && !(v in visited)
                push!(path, v); push!(visited, v); current = v; found = true; break
            end
        end
        !found && break
    end
    return path
end

function enumerate_solutions(arcs, weights, N, source, target, opt_val;
                             symmetry_breaking=false, symmetric_nodes=nothing)
    """
    Enumerates all optimal paths (up to 100) with the same minimum cost.
    
    Args:
        arcs: List of graph arcs
        weights: Dictionary with arc weights
        N: Total number of nodes
        source: Source node
        target: Target node
        opt_val: Optimal value (minimum cost) to match
        symmetry_breaking: (optional) Enables symmetry breaking
        symmetric_nodes: (optional) Symmetric nodes
    
    Returns: Vector of paths (each path is a Vector{Int})
    """
    
    solutions = Vector{Vector{Int}}()
    
    for _ in 1:100
        model, x = build_sp_model(arcs, weights, N, source, target,
                                   symmetry_breaking=symmetry_breaking,
                                   symmetric_nodes=symmetric_nodes,
                                   fixed_obj=opt_val)
        
        for prev_path in solutions
            prev_arcs = [(prev_path[i], prev_path[i+1]) for i in 1:(length(prev_path)-1)]
            @constraint(model, sum(1 - x[(u,v)] for (u,v) in prev_arcs if (u,v) in arcs) >= 1)
        end
        
        optimize!(model)
        termination_status(model) != OPTIMAL && break
        
        path = extract_path(x, arcs, source, target)
        (length(path) < 2 || path[end] != target) && break
        push!(solutions, path)
    end
    
    return solutions
end

enumerate_solutions (generic function with 1 method)

### **Example 1**: Simple Diamond Graph

#### Problem Instance

Consider an undirected graph $G = (V, E)$ with four vertices arranged in a diamond configuration:

\begin{equation*}
\begin{array}{ccc}
 & [2] & \\
 & \diagup \quad \diagdown & \\
[1] & & [4] \\
 & \diagdown \quad \diagup & \\
 & [3] &
\end{array}
\end{equation*}

The vertex set is $V = \{1, 2, 3, 4\}$ and the edge set is $E = \{\{1,2\}, \{1,3\}, \{2,4\}, \{3,4\}\}$. All edges are assigned unit weight $w_e = 1$. We seek the shortest path from source $s = 1$ to target $t = 4$.

This graph exhibits a clear symmetry: nodes $2$ and $3$ occupy structurally equivalent positions. Both are adjacent to the source and to the target, with identical edge weights. Formally, the mapping $\pi: V \to V$ defined by

\begin{equation*}
\pi(1) = 1, \quad \pi(2) = 3, \quad \pi(3) = 2, \quad \pi(4) = 4
\end{equation*}

is a graph automorphism. Under $\pi$, edge $\{1,2\}$ maps to $\{1,3\}$, edge $\{2,4\}$ maps to $\{3,4\}$, and vice versa. Since all edges have equal weight, $\pi$ preserves both structure and weights, confirming $\pi \in \mathrm{Aut}(G)$.

The automorphism group has order $|\mathrm{Aut}(G)| = 2$, generated by the single transposition $(2 \leftrightarrow 3)$. Consequently, the feasible region $\mathcal{F}$ partitions into orbits of size $2$, and we expect the symmetry-breaking constraint $x_{12} \geq x_{13}$ to reduce the number of equivalent optimal solutions by a factor of $2$.

#### Solution Enumeration

In [16]:
edges_1 = [(1,2), (1,3), (2,4), (3,4)]
arcs_1 = vcat(edges_1, [(v,u) for (u,v) in edges_1])
weights_1 = Dict((u,v) => 1.0 for (u,v) in arcs_1)
N_1, source_1, target_1 = 4, 1, 4
symmetric_nodes_1 = [2, 3]

model_1, x_1 = build_sp_model(arcs_1, weights_1, N_1, source_1, target_1)
optimize!(model_1)
opt_val_1 = objective_value(model_1)

solutions_baseline_1 = enumerate_solutions(arcs_1, weights_1, N_1, source_1, target_1, opt_val_1,
                                            symmetry_breaking=false)

solutions_reduced_1 = enumerate_solutions(arcs_1, weights_1, N_1, source_1, target_1, opt_val_1,
                                           symmetry_breaking=true, symmetric_nodes=symmetric_nodes_1)


println("Optimal cost = ", opt_val_1)

println("\nBaseline solutions:")
for (i, p) in enumerate(solutions_baseline_1)
    println("P", i, ": ", join(p, " -> "))
end

println("\nSymmetry-reduced solutions:")
for (i, p) in enumerate(solutions_reduced_1)
    println("P", i, ": ", join(p, " -> "))
end


Optimal cost = 2.0

Baseline solutions:
P1: 1 -> 3 -> 4
P2: 1 -> 2 -> 4

Symmetry-reduced solutions:
P1: 1 -> 2 -> 4


#### Results Analysis

The optimal path cost is $2$, corresponding to any path traversing exactly two edges from source to target.

Without symmetry breaking, the enumeration procedure finds two optimal solutions:

\begin{equation*}
P_1: 1 \to 2 \to 4
\end{equation*}

\begin{equation*}
P_2: 1 \to 3 \to 4
\end{equation*}

These solutions are related by the automorphism $\pi: V \to V$ defined by $\pi(1) = 1$, $\pi(2) = 3$, $\pi(3) = 2$, $\pi(4) = 4$. Under this permutation, path $P_1$ maps to $P_2$ and vice versa, confirming that both belong to the same orbit in the solution space $\mathcal{F}$.

When we introduce the symmetry-breaking constraint $x_{12} \geq x_{13}$, the enumeration yields a single solution:

\begin{equation*}
P_1: 1 \to 2 \to 4
\end{equation*}

The constraint enforces a lexicographic preference for node $2$ over node $3$. Since exactly one outgoing arc from the source can be selected (by flow conservation), setting $x_{13} = 1$ would require $x_{12} \geq 1$, which is infeasible when only one arc can be active. Thus, any path through node $3$ is eliminated, leaving only the canonical representative $P_1$.

The solution space reduction is $|\mathcal{F}| / |\mathcal{F}'| = 2 / 1 = 2$, which matches the size of the automorphism group $|\mathrm{Aut}(G)| = 2$ for this graph.

### **Example 2**: Symmetric Ladder Graph

Consider a larger undirected graph $G = (V, E)$ with a layered ladder structure consisting of $8$ vertices:

\begin{equation*}
\begin{array}{ccccccc}
 & [2] & \longrightarrow & [4] & \longrightarrow & [6] & \\
 & \nearrow & & \downarrow \uparrow & & \searrow & \\
[1] & & & \times & & & [8] \\
 & \searrow & & \uparrow \downarrow & & \nearrow & \\
 & [3] & \longrightarrow & [5] & \longrightarrow & [7] &
\end{array}
\end{equation*}

where the symbol $\times$ denotes full bipartite connectivity between adjacent layers.

The vertex set is $V = \{1, 2, 3, 4, 5, 6, 7, 8\}$, organized into a source node ($1$), three intermediate layers $\{2, 3\}$, $\{4, 5\}$, $\{6, 7\}$, and a target node ($8$). The edge set consists of connections from the source to the first layer, full bipartite connections between consecutive layers, and connections from the last layer to the target. All edges have unit weight $w_e = 1$. We seek the shortest path from $s = 1$ to $t = 8$.

The graph possesses a rich symmetry structure. At each intermediate layer, the two nodes are structurally equivalent: nodes $2$ and $3$ have identical neighborhoods (both connect to node $1$ and to nodes $4$, $5$), and similarly for the pairs $\{4, 5\}$ and $\{6, 7\}$. The automorphism $\pi: V \to V$ defined by

\begin{equation*}
\begin{aligned}
\pi(1) &= 1, & \pi(2) &= 3, & \pi(3) &= 2, & \pi(4) &= 5, \\
\pi(5) &= 4, & \pi(6) &= 7, & \pi(7) &= 6, & \pi(8) &= 8
\end{aligned}
\end{equation*}

simultaneously swaps all upper-lower node pairs while fixing the source and target. This permutation preserves both adjacency and edge weights, confirming $\pi \in \mathrm{Aut}(G)$.

The layered structure creates a multiplicative effect on the number of optimal paths. Any shortest path must traverse exactly four edges: one from source to layer $1$, one from layer $1$ to layer $2$, one from layer $2$ to layer $3$, and one from layer $3$ to target. At the first layer, there are $2$ choices (node $2$ or $3$). Due to the full bipartite connectivity, each subsequent layer also offers $2$ choices. The total number of optimal paths is therefore $2 \times 2 \times 2 = 2^3 = 8$.

All eight paths belong to the same orbit under the automorphism group. A single symmetry-breaking constraint $x_{12} \geq x_{13}$ at the source level will eliminate paths starting with arc $(1, 3)$, reducing the solution count by a factor of $2$. Complete symmetry breaking across all layers would require additional constraints and would isolate a unique canonical path.

#### Problem Instance

In [11]:
function create_ladder(n_branches, n_layers)
    N = 1 + n_branches * n_layers + 1
    source, target = 1, N
    edges = Tuple{Int,Int}[]
    
    first_layer = collect(2 : 1 + n_branches)
    for node in first_layer
        push!(edges, (source, node))
    end
    
    for layer in 1:(n_layers - 1)
        curr_start = 2 + (layer - 1) * n_branches
        next_start = 2 + layer * n_branches
        curr_layer = collect(curr_start : curr_start + n_branches - 1)
        next_layer = collect(next_start : next_start + n_branches - 1)
        
        for u in curr_layer, v in next_layer
            push!(edges, (u, v))
        end
    end
    
    last_start = 2 + (n_layers - 1) * n_branches
    last_layer = collect(last_start : last_start + n_branches - 1)
    for node in last_layer
        push!(edges, (node, target))
    end
    
    arcs = vcat(edges, [(v, u) for (u, v) in edges])
    weights = Dict((u, v) => 1.0 for (u, v) in arcs)
    
    return arcs, weights, N, source, target, first_layer
end

arcs_2, weights_2, N_2, source_2, target_2, symmetric_nodes_2 = create_ladder(2, 3)

println("N = ", N_2, ", |E| = ", length(arcs_2) ÷ 2)
println("Symmetric nodes: ", symmetric_nodes_2)


N = 8, |E| = 12
Symmetric nodes: [2, 3]


#### Solution Enumeration

In [15]:
# Find optimal value
model_2, x_2 = build_sp_model(arcs_2, weights_2, N_2, source_2, target_2)
optimize!(model_2)
opt_val_2 = objective_value(model_2)

# Enumerate solutions
solutions_baseline_2 = enumerate_solutions(arcs_2, weights_2, N_2, source_2, target_2, opt_val_2,
                                            symmetry_breaking=false)

solutions_reduced_2 = enumerate_solutions(arcs_2, weights_2, N_2, source_2, target_2, opt_val_2,
                                           symmetry_breaking=true, symmetric_nodes=symmetric_nodes_2)


println("Optimal value = ", opt_val_2)
println("Expected paths = ", 2^3)

println("\nBaseline solutions:")
for (i, p) in enumerate(solutions_baseline_2)
    println("P", i, ": ", join(p, " -> "))
end

println("\nSymmetry-reduced solutions:")
for (i, p) in enumerate(solutions_reduced_2)
    println("P", i, ": ", join(p, " -> "))
end

Optimal value = 4.0
Expected paths = 8

Baseline solutions:
P1: 1 -> 3 -> 4 -> 6 -> 8
P2: 1 -> 2 -> 5 -> 7 -> 8
P3: 1 -> 2 -> 4 -> 7 -> 8
P4: 1 -> 2 -> 4 -> 6 -> 8
P5: 1 -> 3 -> 5 -> 6 -> 8
P6: 1 -> 3 -> 5 -> 7 -> 8
P7: 1 -> 3 -> 4 -> 7 -> 8
P8: 1 -> 2 -> 5 -> 6 -> 8

Symmetry-reduced solutions:
P1: 1 -> 2 -> 5 -> 7 -> 8
P2: 1 -> 2 -> 4 -> 7 -> 8
P3: 1 -> 2 -> 5 -> 6 -> 8
P4: 1 -> 2 -> 4 -> 6 -> 8


#### Results Analysis

The optimal path cost is $4$, corresponding to paths traversing four edges: one from source to layer $1$, one between each consecutive layer, and one from layer $3$ to target.

The graph structure creates a multiplicative symmetry: at each of the three layers, there are $2$ symmetric choices (upper or lower branch). This yields $2^3 = 8$ optimal paths, all with identical cost. Without symmetry breaking, the enumeration confirms all $8$ solutions:

\begin{equation*}
\begin{aligned}
P_1 &: 1 \to 2 \to 4 \to 6 \to 8 \\
P_2 &: 1 \to 2 \to 4 \to 7 \to 8 \\
P_3 &: 1 \to 2 \to 5 \to 6 \to 8 \\
P_4 &: 1 \to 2 \to 5 \to 7 \to 8 \\
P_5 &: 1 \to 3 \to 4 \to 6 \to 8 \\
P_6 &: 1 \to 3 \to 4 \to 7 \to 8 \\
P_7 &: 1 \to 3 \to 5 \to 6 \to 8 \\
P_8 &: 1 \to 3 \to 5 \to 7 \to 8
\end{aligned}
\end{equation*}

The automorphism group of this graph is generated by the permutation $\pi = (2 \leftrightarrow 3)(4 \leftrightarrow 5)(6 \leftrightarrow 7)$, which simultaneously swaps all upper-lower node pairs. Paths $P_1$ through $P_4$ (starting with $1 \to 2$) are mapped to paths $P_5$ through $P_8$ (starting with $1 \to 3$) under $\pi$.

Introducing the symmetry-breaking constraint $x_{12} \geq x_{13}$ at the source level eliminates all paths beginning with arc $(1, 3)$. The reduced enumeration yields $4$ solutions:

\begin{equation*}
\begin{aligned}
P_1 &: 1 \to 2 \to 4 \to 6 \to 8 \\
P_2 &: 1 \to 2 \to 4 \to 7 \to 8 \\
P_3 &: 1 \to 2 \to 5 \to 6 \to 8 \\
P_4 &: 1 \to 2 \to 5 \to 7 \to 8
\end{aligned}
\end{equation*}

The reduction factor is $8 / 4 = 2$, corresponding to breaking the symmetry at the first layer only. The remaining $4$ solutions still exhibit symmetry at deeper layers: $P_1 \leftrightarrow P_2$ under $(6 \leftrightarrow 7)$ and $P_3 \leftrightarrow P_4$ under the same permutation.

To achieve complete symmetry breaking and retain a unique canonical solution, one would impose additional constraints at each layer:

\begin{equation*}
x_{12} \geq x_{13}, \quad x_{24} \geq x_{25}, \quad x_{46} \geq x_{47}, \quad x_{56} \geq x_{57}
\end{equation*}

This hierarchical application of lexicographic constraints would reduce the solution space to $|\mathcal{F}'| = 1$, selecting the unique path $P_1: 1 \to 2 \to 4 \to 6 \to 8$ as the canonical representative of the entire orbit $\mathrm{Orb}(P_1) = \mathcal{F}$.

## **Discussion**

The results obtained in this notebook illustrate the effect of lexicographic symmetry-breaking constraints on the shortest path problem formulated as an Integer Linear Program.

In Example 1, the diamond graph with $4$ vertices exhibited a single non-trivial automorphism that swaps nodes $2$ and $3$. Without symmetry breaking, the enumeration procedure identified $2$ optimal solutions with cost $2$. The introduction of the constraint $x_{12} \geq x_{13}$ reduced the solution space to a single canonical path. The reduction factor of $|\mathcal{F}|/|\mathcal{F}'| = 2$ coincides with the order of the automorphism group $|\mathrm{Aut}(G)| = 2$, confirming that the symmetry was fully broken.

In Example 2, the symmetric ladder graph with $8$ vertices presented a more complex structure with $3$ layers of symmetric node pairs. The enumeration without constraints yielded $2^3 = 8$ optimal solutions, each with cost $4$. Applying the symmetry-breaking constraint $x_{12} \geq x_{13}$ at the source level reduced the count to $4$ solutions. The reduction factor of $2$ corresponds to eliminating the symmetric choice at the first layer only.

In both cases, the optimal objective value was preserved. This observation is consistent with the theoretical properties of valid symmetry-breaking schemes: *completeness*, ensuring that at least one solution from each equivalence class remains feasible, and *soundness*, guaranteeing that no optimal solutions are excluded from the reduced space.

It should be noted that the symmetry-breaking strategy employed in Example 2 is partial. The $4$ remaining solutions still exhibit pairwise equivalence under the permutations $(4 \leftrightarrow 5)$ and $(6 \leftrightarrow 7)$. A complete treatment would require imposing additional constraints at each symmetric layer, as indicated in the results. The systematic construction of such hierarchical symmetry-breaking schemes, as well as the extension of these techniques to other combinatorial optimization problems, constitutes a direction for future work.